# 🚀 Hello Agent OS

> **Your first governed agent in under 5 minutes.**

## Learning Objectives

By the end of this notebook, you will:
1. Install Agent OS
2. Create a basic governed agent
3. Understand kernel vs. user space
4. See policy enforcement in action

---

## Step 1: Install Agent OS

Run this cell to install the package:

In [ ]:
!pip install agent-os --quiet

## Step 2: Import and Initialize

The `KernelSpace` is the core of Agent OS. It intercepts all agent actions and checks them against policies.

In [ ]:
from agent_os import KernelSpace

# Create a kernel with strict policy enforcement
kernel = KernelSpace(policy="strict")

print("✅ Kernel initialized with 'strict' policy")
print(f"   Policy mode: {kernel.policy.mode}")

## Step 3: Create Your First Agent

Use `@kernel.register` to wrap your function with kernel governance.

In [ ]:
@kernel.register
async def hello_agent(task: str):
    """
    A simple agent that processes tasks.
    
    This function runs in USER SPACE.
    The kernel intercepts its actions before execution.
    """
    result = f"Hello! I processed your task: {task}"
    return result

print("✅ Agent registered with kernel")

## Step 4: Execute with Governance

Use `kernel.execute()` to run your agent through the policy engine.

In [ ]:
import asyncio

# Execute the agent
result = await kernel.execute(hello_agent, "Summarize today's news")

print(f"\n📤 Result: {result}")

## 🎉 Congratulations!

You just ran your first governed agent. Here's what happened:

```
┌─────────────────────────────────────────────────────────┐
│              USER SPACE                                 │
│   hello_agent() executed here                           │
├─────────────────────────────────────────────────────────┤
│              KERNEL SPACE                               │
│   ✓ Action checked against 'strict' policy              │
│   ✓ No violations detected                              │
│   ✓ Result returned to user space                       │
└─────────────────────────────────────────────────────────┘
```

---

## Step 5: See Policy Enforcement

Let's create an agent that tries to do something blocked by the `strict` policy:

In [ ]:
@kernel.register
async def dangerous_agent(task: str):
    """An agent that tries to write to files (blocked by strict policy)."""
    import os
    
    # This will be intercepted by the kernel!
    with open("/tmp/secret.txt", "w") as f:
        f.write("sensitive data")
    
    return "File written successfully"

print("⚠️  Agent registered - but it will be blocked...")

In [ ]:
# Try to execute the dangerous agent
try:
    result = await kernel.execute(dangerous_agent, "Write some data")
    print(f"Result: {result}")
except Exception as e:
    print(f"🛑 BLOCKED: {e}")
    print("\nThe kernel prevented the file write before it executed!")

## Key Concepts

| Concept | Description |
|---------|-------------|
| **User Space** | Where your agent code runs |
| **Kernel Space** | Where policies are enforced |
| **`@kernel.register`** | Wraps a function for governance |
| **`kernel.execute()`** | Runs agent through policy engine |
| **Policy** | Rules about what agents can/can't do |
| **SIGKILL** | Signal to terminate on violation |

---

## Step 6: Add an LLM (Optional)

Here's how to wrap a real LLM call with governance:

In [ ]:
# Uncomment and run if you have an OpenAI API key

# import os
# os.environ["OPENAI_API_KEY"] = "your-key-here"

# from openai import OpenAI
# client = OpenAI()

# @kernel.register
# async def smart_agent(task: str):
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[{"role": "user", "content": task}]
#     )
#     return response.choices[0].message.content

# result = await kernel.execute(smart_agent, "What is 2+2?")
# print(result)

## Next Steps

Continue learning with these notebooks:

| Notebook | Description | Time |
|----------|-------------|------|
| [02-episodic-memory-demo](02-episodic-memory-demo.ipynb) | Agent memory that persists | 15 min |
| [03-time-travel-debugging](03-time-travel-debugging.ipynb) | Replay and debug decisions | 20 min |
| [04-cross-model-verification](04-cross-model-verification.ipynb) | Detect hallucinations | 15 min |
| [05-multi-agent-coordination](05-multi-agent-coordination.ipynb) | Trust between agents | 20 min |
| [06-policy-engine](06-policy-engine.ipynb) | Deep dive into policies | 15 min |

---

## Summary

```python
# The 4-line pattern for governed agents:

from agent_os import KernelSpace

kernel = KernelSpace(policy="strict")

@kernel.register
async def my_agent(task): ...

await kernel.execute(my_agent, "task")
```